In [87]:
# !pip install opensmile
# try:
#     from importlib import metadata
# except ImportError:
#     # Running on pre-3.8 Python; use importlib-metadata package
#     import importlib_metadata as metadata  # type: ignore
# import opensmile

In [85]:
data_dir='./data/voice/processed/'
import os
from scipy.io.wavfile import read
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from PIL import Image
import scipy.misc
import cv2
import librosa
from tqdm import tqdm
import numpy as np
import librosa
import librosa.display
import os
import glob
import scipy
import matplotlib.pyplot as plt
import librosa
import IPython
import cv2
from IPython.display import Audio
import soundfile as sf
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
cmap = plt.get_cmap('inferno')
import python_speech_features as mfcc
data=[]
label=[]
file = './data/voice/final_val.csv'
df = pd.read_csv(file)
df = df.drop(['Unnamed: 0'],axis=1)
df = df.drop_duplicates('ID')
df

,ID,Age,Gender,Voice Handicap Index (VHI) Score,Reflux Symptom Index (RSI) Score,Smoker,Alcohol consumption,Amount of water's litres drink every day,Tomatoes,Coffee,Chocolate,Soft cheese,Citrus fruits,Diagnosis
0,voice001,32,1,15,5,0,1,1,1,3,0,1,1,1
1,voice002,55,1,17,12,1,1,0,1,1,1,3,3,0
2,voice003,34,1,42,26,0,1,1,1,3,1,3,0,1
3,voice004,28,0,20,9,1,1,1,1,4,1,3,1,2
4,voice005,54,0,39,23,0,1,1,1,2,1,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,voice204,59,0,0,7,0,0,2,1,1,1,0,0,0
204,voice205,30,0,62,19,1,1,1,1,4,3,1,1,1
205,voice206,22,0,0,6,0,0,2,1,1,1,1,0,0
206,voice207,30,0,0,26,0,0,2,1,2,1,1,1,0


In [86]:
clean_data = np.load('clean_data.npy',allow_pickle=True)
clean_labels = np.load('clean_labels.npy')
noise_added = np.load('noise_added.npy')
clean_pad = np.load('clean_pad.npy')
denoised = np.load('denoised_audio.npy')

In [153]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)

In [155]:
# len(smile.feature_names)

988

In [157]:

name = 'openSmile_features_emo.csv'
try:
    os.remove(name)
except:
    pass

from collections import defaultdict
dat = defaultdict(list)

for i in tqdm(denoised):
    scipy.io.wavfile.write("karplus.wav", 8000, i)
    result_df = smile.process_file('karplus.wav')
    for j in range(len(result_df.values[0])):
#         if result_df.columns[j] not in dat.keys():
#             dat[result_df.columns[j]]=[]
        dat[result_df.columns[j]].append(result_df.values[0][j])


F_df = pd.DataFrame(dat)
# result_df.columns[0]
F_df.to_csv(name)
F_df

100%|██████████████████████████████████████████████████████████████████████████████| 2060/2060 [01:43<00:00, 19.85it/s]


,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,pcm_intensity_sma_linregerrQ,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,0.000323,1.868601e-07,0.000323,35.0,7.0,0.000082,4.070177e-06,-0.000003,0.000066,7.073674e-09,...,259.528931,17.036528,1.763549,5.707826,-3.812931,-0.211903,0.000000,3.601028,0.211903,3.812931
1,0.000289,2.755640e-05,0.000261,14.0,40.0,0.000119,-1.725775e-06,0.000155,0.000064,5.368969e-09,...,13.119859,5.134985,2.093541,6.197203,0.012897,0.086481,1.947129,0.073584,1.860649,1.934233
2,0.000467,1.918908e-05,0.000447,23.0,27.0,0.000145,8.170517e-07,0.000128,0.000098,1.594286e-08,...,12.268421,5.028275,2.072419,6.129059,0.038212,0.218547,2.213107,0.180335,1.994560,2.174895
3,0.000222,1.963740e-05,0.000202,12.0,0.0,0.000096,1.091328e-06,0.000073,0.000047,3.056506e-09,...,13.173339,5.124740,2.079568,6.141874,-0.009866,0.179552,1.969452,0.189417,1.789900,1.979317
4,0.000374,3.397455e-05,0.000340,33.0,39.0,0.000111,1.307143e-06,0.000083,0.000064,7.548260e-09,...,12.798785,5.129630,2.077530,6.134487,0.006248,0.095065,1.969452,0.088817,1.874387,1.963203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,0.000099,4.329313e-06,0.000095,37.0,19.0,0.000044,-9.343675e-08,0.000046,0.000024,7.497903e-10,...,28.044930,7.482595,2.102696,6.211283,0.000057,0.001172,2.647508,0.001115,2.646336,2.647451
2056,0.000147,4.871821e-06,0.000142,39.0,42.0,0.000052,3.233295e-07,0.000045,0.000033,1.594110e-09,...,28.352409,7.569348,2.030229,5.908770,-0.138606,-0.006981,2.769403,0.131625,2.776384,2.908009
2057,0.000182,5.995739e-06,0.000176,19.0,0.0,0.000050,5.739194e-07,0.000038,0.000030,1.691905e-09,...,27.475727,7.444227,2.088284,6.150899,-0.015308,0.351170,2.694252,0.366479,2.343082,2.709560
2058,0.000151,6.534719e-06,0.000144,25.0,36.0,0.000066,-3.732638e-07,0.000074,0.000046,2.527386e-09,...,27.137070,7.384679,2.095805,6.215002,-0.013980,0.246770,2.802116,0.260750,2.555346,2.816096


In [110]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
output_matrix = to_categorical(clean_labels, num_classes = 4)
data = './scaled_openSmile_features.csv'
seed = 9
np.random.seed(seed)

# load pima indians dataset
dataset =  pd.read_csv('./openSmile_features.csv')
dataset = dataset.drop(['Unnamed: 0'],axis=1)

(X_train, X_test, y_train, y_test) = train_test_split(dataset, output_matrix, test_size=0.2, random_state=seed,shuffle=True)
# X_train=X_train.values
# y_train=y_train.values
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1648, 6373) (412, 6373) (1648, 4) (412, 4)


In [89]:
# dataset

In [148]:
from sklearn.tree import DecisionTreeClassifier,
from sklearn.model_selection import KFold, cross_val_score
import sklearn
clf = DecisionTreeClassifier(max_depth =None, random_state = 42)

# k_folds = KFold(n_splits = 5)

# scores = cross_val_score(clf, dataset, clean_labels, cv = k_folds)
clf.fit(X_train, np.argmax(y_train,axis=-1))
feat = clf.feature_importances_
y_pred = clf.predict(X_test)

target_names = ['healthy', 'hyperkinetic dysphonia', 'hypokinetic dysphonia','reflux laryngitis']

# from s
print(sklearn.metrics.classification_report(np.argmax(y_test,axis=-1), y_pred, labels=[0,1,2,3], target_names=target_names))

TypeError: load() missing 1 required positional argument: 'Loader'

In [152]:
# print(dataset[dataset.columns[list(feat).index(max(feat))]])
for i in (dataset.columns):
    if 'l' in i:
        print(i)

audspec_lengthL1norm_sma_range
audspec_lengthL1norm_sma_maxPos
audspec_lengthL1norm_sma_minPos
audspec_lengthL1norm_sma_quartile1
audspec_lengthL1norm_sma_quartile2
audspec_lengthL1norm_sma_quartile3
audspec_lengthL1norm_sma_iqr1-2
audspec_lengthL1norm_sma_iqr2-3
audspec_lengthL1norm_sma_iqr1-3
audspec_lengthL1norm_sma_percentile1.0
audspec_lengthL1norm_sma_percentile99.0
audspec_lengthL1norm_sma_pctlrange0-1
audspec_lengthL1norm_sma_stddev
audspec_lengthL1norm_sma_skewness
audspec_lengthL1norm_sma_kurtosis
audspec_lengthL1norm_sma_meanSegLen
audspec_lengthL1norm_sma_maxSegLen
audspec_lengthL1norm_sma_minSegLen
audspec_lengthL1norm_sma_segLenStddev
audspec_lengthL1norm_sma_upleveltime25
audspec_lengthL1norm_sma_upleveltime50
audspec_lengthL1norm_sma_upleveltime75
audspec_lengthL1norm_sma_upleveltime90
audspec_lengthL1norm_sma_risetime
audspec_lengthL1norm_sma_leftctime
audspec_lengthL1norm_sma_lpgain
audspec_lengthL1norm_sma_lpc0
audspec_lengthL1norm_sma_lpc1
audspec_lengthL1norm_sma_l

audSpec_Rfilt_sma_de[21]_kurtosis
audSpec_Rfilt_sma_de[21]_meanSegLen
audSpec_Rfilt_sma_de[21]_maxSegLen
audSpec_Rfilt_sma_de[21]_minSegLen
audSpec_Rfilt_sma_de[21]_segLenStddev
audSpec_Rfilt_sma_de[21]_upleveltime25
audSpec_Rfilt_sma_de[21]_upleveltime50
audSpec_Rfilt_sma_de[21]_upleveltime75
audSpec_Rfilt_sma_de[21]_upleveltime90
audSpec_Rfilt_sma_de[21]_risetime
audSpec_Rfilt_sma_de[21]_leftctime
audSpec_Rfilt_sma_de[21]_lpgain
audSpec_Rfilt_sma_de[21]_lpc0
audSpec_Rfilt_sma_de[21]_lpc1
audSpec_Rfilt_sma_de[21]_lpc2
audSpec_Rfilt_sma_de[21]_lpc3
audSpec_Rfilt_sma_de[21]_lpc4
audSpec_Rfilt_sma_de[22]_range
audSpec_Rfilt_sma_de[22]_maxPos
audSpec_Rfilt_sma_de[22]_minPos
audSpec_Rfilt_sma_de[22]_quartile1
audSpec_Rfilt_sma_de[22]_quartile2
audSpec_Rfilt_sma_de[22]_quartile3
audSpec_Rfilt_sma_de[22]_iqr1-2
audSpec_Rfilt_sma_de[22]_iqr2-3
audSpec_Rfilt_sma_de[22]_iqr1-3
audSpec_Rfilt_sma_de[22]_percentile1.0
audSpec_Rfilt_sma_de[22]_percentile99.0
audSpec_Rfilt_sma_de[22]_pctlrange0-1
au

audSpec_Rfilt_sma[15]_linregc2
audSpec_Rfilt_sma[15]_linregerrQ
audSpec_Rfilt_sma[15]_qregc1
audSpec_Rfilt_sma[15]_qregc2
audSpec_Rfilt_sma[15]_qregc3
audSpec_Rfilt_sma[15]_qregerrQ
audSpec_Rfilt_sma[15]_centroid
audSpec_Rfilt_sma[16]_amean
audSpec_Rfilt_sma[16]_flatness
audSpec_Rfilt_sma[16]_rqmean
audSpec_Rfilt_sma[16]_meanPeakDist
audSpec_Rfilt_sma[16]_peakDistStddev
audSpec_Rfilt_sma[16]_peakRangeAbs
audSpec_Rfilt_sma[16]_peakRangeRel
audSpec_Rfilt_sma[16]_peakMeanAbs
audSpec_Rfilt_sma[16]_peakMeanMeanDist
audSpec_Rfilt_sma[16]_peakMeanRel
audSpec_Rfilt_sma[16]_minRangeRel
audSpec_Rfilt_sma[16]_meanRisingSlope
audSpec_Rfilt_sma[16]_stddevRisingSlope
audSpec_Rfilt_sma[16]_meanFallingSlope
audSpec_Rfilt_sma[16]_stddevFallingSlope
audSpec_Rfilt_sma[16]_linregc1
audSpec_Rfilt_sma[16]_linregc2
audSpec_Rfilt_sma[16]_linregerrQ
audSpec_Rfilt_sma[16]_qregc1
audSpec_Rfilt_sma[16]_qregc2
audSpec_Rfilt_sma[16]_qregc3
audSpec_Rfilt_sma[16]_qregerrQ
audSpec_Rfilt_sma[16]_centroid
audSpec_Rfilt_

In [91]:
# y_pred = clf.predict(X_test)

# target_names = ['healthy', 'hyperkinetic dysphonia', 'hypokinetic dysphonia','reflux laryngitis']
# import sklearn
# import seaborn as sns
# # from s
# print(sklearn.metrics.classification_report(y_test, y_pred, labels=[0,1,2,3], target_names=target_names))
# print(sklearn.metrics.confusion_matrix(y_test, y_pred, labels=[0,1,2,3]))


# def plot_conf_mat(Ground_Truth_Labels, Predictions):
#   confusion_matrix_raw = sklearn.metrics.confusion_matrix(Ground_Truth_Labels, Predictions, normalize=None)
#   confusion_matrix_norm = sklearn.metrics.confusion_matrix(Ground_Truth_Labels, Predictions, normalize='true')
#   shape = confusion_matrix_raw.shape
#   data = np.asarray(confusion_matrix_raw, dtype=int)
#   text = np.asarray(confusion_matrix_norm, dtype=float)
#   annots = (np.asarray(["{0:.2f} ({1:.0f})".format(text, data) for text, data in zip(text.flatten(), data.flatten())])).reshape(shape[0],shape[1])
#   fig = plt.figure(figsize=(20, 10))
#   sns.heatmap(confusion_matrix_norm, cmap='YlGnBu', annot=annots, fmt='', xticklabels=target_names, yticklabels=target_names)
#   plt.title('Confusion Matrix', fontsize=25)
#   plt.xlabel("Predicted", fontsize=15)
#   plt.ylabel("Actual", fontsize=15)
# #
# plot_conf_mat(y_test, y_pred)

In [113]:
# tf.keras.backend.clear_session()
# cl_model = tf.keras.models.Sequential([
#     tf.keras.layers.BatchNormalization(input_shape=[6373]),
#     tf.keras.layers.Dense(5000,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(3500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(2500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1750, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1250, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(750, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(325, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(4, activation='softmax')
#     ])
# cl_model.summary()
# old =0
# class myCallback(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,logs={}):
#             global old
#             if(logs.get('val_acc') > old):
# #                 cl_model.save('openSmile_model.h5',overwrite=True)
#                 old = logs.get('val_acc')
#                 print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
#                       round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
#                       round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
# saver =myCallback()
# cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
#                                                                               tf.keras.metrics.SensitivityAtSpecificity(0.5),
#                                                                               tf.keras.metrics.SpecificityAtSensitivity(0.5),
#                                                                              tf.keras.metrics.Precision()])
# history = cl_model.fit(x=X_train,y=y_train,
#                               epochs=5000,
#                               verbose=0,
#                               batch_size=4096*2,
#                               validation_data=(X_test,y_test),callbacks=[saver])

In [114]:
# tf.keras.backend.clear_session()
# cl_model = tf.keras.models.Sequential([
#     tf.keras.layers.BatchNormalization(input_shape=[6374]),
#     tf.keras.layers.Dense(5000,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(3500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(2500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1750, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1250, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(750, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(500, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(325, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(200, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(100, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(4, activation='softmax')
#     ])
# cl_model.summary()
# old =0
# class myCallback(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,logs={}):
#             global old
#             if(logs.get('val_acc') > old):
# #                 cl_model.save('openSmile_model.h5',overwrite=True)
#                 old = logs.get('val_acc')
#                 print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
#                       round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
#                       round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
# saver =myCallback()
# cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
#                                                                               tf.keras.metrics.SensitivityAtSpecificity(0.5),
#                                                                               tf.keras.metrics.SpecificityAtSensitivity(0.5),
#                                                                              tf.keras.metrics.Precision()])
# history = cl_model.fit(x=X_train,y=y_train,
#                               epochs=5000,
#                               verbose=0,
#                               batch_size=4096*2,
#                               validation_data=(X_test,y_test),callbacks=[saver])

In [119]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=[6373]),
    tf.keras.layers.Dense(5000,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1250, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(325, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(4, activation='softmax')
    ])
cl_model.summary()
old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
#                 cl_model.save('openSmile_model.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])
history = cl_model.fit(x=X_train,y=y_train,
                              epochs=5000,
                              verbose=0,
                              batch_size=4096*2,
                              validation_data=(X_test,y_test),callbacks=[saver])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 6373)              25492     
_________________________________________________________________
dense (Dense)                (None, 5000)              31870000  
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5000)              20000     
_________________________________________________________________
dense_1 (Dense)              (None, 3500)              17503500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 3500)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3500)              1

Epoch= 1592 acc =  0.5631 sensitivity 0.6092 specificity= 0.8754 precision= 0.5599
Epoch= 1605 acc =  0.5655 sensitivity 0.6019 specificity= 0.8722 precision= 0.5645
Epoch= 1916 acc =  0.5752 sensitivity 0.6117 specificity= 0.8803 precision= 0.5742


KeyboardInterrupt: 

In [121]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(5000,activation='relu',input_shape=[6373]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5000,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1250, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(325, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
    ])
cl_model.summary()
old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
#                 cl_model.save('openSmile_model.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])
history = cl_model.fit(x=X_train,y=y_train,
                              epochs=5000,
                              verbose=0,
                              batch_size=4096*2,
                              validation_data=(X_test,y_test),callbacks=[saver])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5000)              31870000  
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 5000)              20000     
_________________________________________________________________
dense_1 (Dense)              (None, 3500)              17503500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 3500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              17505000  
_________________________________________________________________
dropout_2 (Dropout)          (None, 5000)              0

KeyboardInterrupt: 

In [20]:
# import pyforest
# import warnings
# warnings.filterwarnings("ignore")
# from sklearn import metrics
# from sklearn.metrics import accuracy_score

# import lazypredict
# from lazypredict.Supervised import LazyClassifier


TypeError: load() missing 1 required positional argument: 'Loader'

In [136]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(5000,activation='relu',input_shape=[6373]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1250, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(750, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(325, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(4, activation='softmax')
    ])
cl_model.summary()
old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
#                 cl_model.save('openSmile_model.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4))
#         ,'sensitivity',
#                       round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
#                       round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc'])
#                  ,
#                                                                               tf.keras.metrics.SensitivityAtSpecificity(0.5),
#                                                                               tf.keras.metrics.SpecificityAtSensitivity(0.5),
#                                                                              tf.keras.metrics.Precision()])
history = cl_model.fit(x=X_train,y=y_train,
                              epochs=5000,
                              verbose=1,
                              batch_size=4096*2,
                              validation_data=(X_test,y_test),callbacks=[saver])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5000)              31870000  
_________________________________________________________________
dropout (Dropout)            (None, 5000)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 5000)              20000     
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              10002000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2000)              8000      
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2

1/1 [==============================] - 0s 72ms/step - loss: 1.5479 - acc: 0.2967 - val_loss: 36.2057 - val_acc: 0.3131
Epoch 25/5000
1/1 [==============================] - 0s 60ms/step - loss: 1.6305 - acc: 0.2737 - val_loss: 37.0216 - val_acc: 0.3131
Epoch 26/5000
1/1 [==============================] - 0s 53ms/step - loss: 1.5661 - acc: 0.2773 - val_loss: 37.6243 - val_acc: 0.3131
Epoch 27/5000
1/1 [==============================] - 0s 56ms/step - loss: 1.5541 - acc: 0.2888 - val_loss: 37.0506 - val_acc: 0.3131
Epoch 28/5000
1/1 [==============================] - 0s 54ms/step - loss: 1.5438 - acc: 0.2743 - val_loss: 36.6455 - val_acc: 0.3131
Epoch 29/5000
1/1 [==============================] - 0s 61ms/step - loss: 1.5700 - acc: 0.2773 - val_loss: 36.3042 - val_acc: 0.3131
Epoch 30/5000
1/1 [==============================] - 0s 53ms/step - loss: 1.5645 - acc: 0.2894 - val_loss: 36.3569 - val_acc: 0.3131
Epoch 31/5000
1/1 [==============================] - 0s 63ms/step - loss: 1.5235 - 

KeyboardInterrupt: 

In [134]:
# from sklearn.neural_network import MLPClassifier

# #Initializing the MLPClassifier
# classifier = MLPClassifier(hidden_layer_sizes=(1500,1000,500),verbose=True, max_iter=300,
#                        activation = 'relu',solver='adam',
#                            random_state=1)

In [135]:
# classifier.fit(X_train, y_train)


# #Predicting y for X_val
# y_pred = classifier.predict(X_test)

Iteration 1, loss = 54.85401088
Iteration 2, loss = 52.18570168
Iteration 3, loss = 53.30108822


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [133]:
target_names = ['healthy', 'hyperkinetic dysphonia', 'hypokinetic dysphonia','reflux laryngitis']
import sklearn

print(sklearn.metrics.classification_report(y_test, y_pred, labels=[0,1,2,3], target_names=target_names))

                        precision    recall  f1-score   support

               healthy       0.29      1.00      0.45       121
hyperkinetic dysphonia       0.41      0.63      0.50       129
 hypokinetic dysphonia       0.00      0.00      0.00        77
     reflux laryngitis       0.22      0.84      0.35        85

             micro avg       0.29      0.66      0.41       412
             macro avg       0.23      0.62      0.33       412
          weighted avg       0.26      0.66      0.36       412
           samples avg       0.29      0.66      0.40       412



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
history = cl_model.fit(x=X_train,y=y_train,
                              epochs=1000,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver])

In [14]:
# cl_model.save('final_voice_model_new.h5',overwrite=True)

In [82]:
# y_train.shape,y_test.shape,np.array(X_test).shape,np.array(X_train).shape

In [11]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (7),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,mode='auto', min_delta=0.01,min_lr=0.0001)

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver,red_lr])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1018, 256)         14592     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 509, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 505, 256)          327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 252, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 248, 256)          327936    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 256)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 122, 256)          1

In [12]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (7),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (7),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,mode='auto', min_delta=0.01,min_lr=0.0001)

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver])#,red_lr])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1018, 256)         14592     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 509, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 503, 256)          459008    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 251, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 247, 512)          655872    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 123, 512)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 119, 512)          1

In [17]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (7),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,mode='auto', min_delta=0.01,min_lr=0.0001)

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver])#,red_lr])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1018, 256)         14592     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 509, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 505, 256)          327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 252, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 248, 512)          655872    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 512)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 122, 512)          7

In [18]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (7),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,mode='auto', min_delta=0.01,min_lr=0.0001)

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver,red_lr])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1018, 256)         14592     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 509, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 505, 256)          327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 252, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 248, 512)          655872    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 512)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 122, 512)          7

In [19]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (5),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(512, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
red_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1,mode='auto', min_delta=0.01,min_lr=0.0001)

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print('Epoch=',epoch,"acc = ",round(old,4),'sensitivity',
                      round(logs.get('val_sensitivity_at_specificity'),4),'specificity=',
                      round(logs.get('val_specificity_at_sensitivity'),4),'precision=',round(logs.get('val_precision'),4))
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc',
                                                                              tf.keras.metrics.SensitivityAtSpecificity(0.5),
                                                                              tf.keras.metrics.SpecificityAtSensitivity(0.5),
                                                                             tf.keras.metrics.Precision()])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver])#,red_lr])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1020, 256)         10496     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 510, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 508, 256)          196864    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 254, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 512)          655872    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 125, 512)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 123, 512)          7

In [34]:
tf.keras.backend.clear_session()
cl_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, (7),activation='relu', input_shape=(1024,8)),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (5),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, (3),activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LSTM(200,return_sequences=True),
#         tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same",activity_regularizer=tf.keras.regularizers.L1L2(l1=0.3, l2=0.01)),
#     #     tf.keras.layers.Flatten(),
#     #     #tf.keras.layers.BatchNormalization(),
#     #     tf.keras.layers.Dropout(0.2),
#     #     tf.keras.layers.Dense(8, activation='relu'),
#     #     tf.keras.layers.Dense(1, activation='sigmoid')
#     # tf.keras.layers.LocallyConnected1D(64, (3),activation='relu',padding="valid", input_shape=(4, 4096)),
#     tf.keras.layers.MaxPooling1D(2),
    # tf.keras.layers.LSTM(128,return_sequences=True),
    #tf.keras.layers.LocallyConnected1D(128, (1),activation='relu'),
    # tf.keras.layers.Conv1D(128, (3),activation='relu',padding='same'),
    #tf.keras.layers.MaxPooling1D(2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Input(shape=(1024,8)),
    tf.keras.layers.Flatten(),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
    ])

old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                cl_model.save('voice_model_new.h5',overwrite=True)
                old = logs.get('val_acc')
                print("\n new best found threshold now = ",round(old,4),'Epoch=',epoch)
                
saver =myCallback()
cl_model.summary()
cl_model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc'])

history = cl_model.fit(x=X_train,y=y_train,
                              epochs=500,
                              verbose=0,
                              batch_size=128,
                              validation_data=(X_test,y_test),callbacks=[saver])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1018, 256)         14592     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 509, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 505, 256)          327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 252, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 248, 256)          327936    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 256)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 122, 256)          1

In [ ]:
cnn_model_input = tf.keras.layers.Input(shape=(512, 512, 3))
x=tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(cnn_model_input)
x=tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2, 2)(x)
x=tf.keras.layers.Conv2D(4, (3, 3), activation='relu')(x)
x=tf.keras.layers.Conv2D(4, (3, 3), activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2, 2)(x)
x=tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
x=tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
x=tf.keras.layers.MaxPooling2D(2, 2)(x)
cnn_model_out=tf.keras.layers.Flatten()(x)
cnn_model_out=tf.keras.layers.Dense(8)(x)

lin_model_input = tf.keras.layers.Input(shape=(12))
lin_model_out=tf.keras.layers.Dense(8)(lin_model_input)

merge = tf.keras.layers.Concatenate(axis=-1)([lin_model_out,cnn_model_out])
output = tf.keras.layers.Dense(4,activation='softmax')(merge)

model = tf.keras.Model(inputs=[cnn_model_input,lin_model_input], outputs=output, name="final_model")
# lin_trainX.shape,lin_trainY.shape,lin_valX.shape,lin_valY.shape
# cnn_trainX.shape,cnn_trainY.shape,cnn_valX.shape,cnn_valY.shape
model.summary()
model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(x=[cnn_trainX,lin_trainX],y=cnn_trainY,
                              epochs=20,
                              verbose=1,
                              validation_data=([cnn_valX,lin_valX],cnn_valY))

In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
i
classification_models = [
    KNeighborsClassifier(4),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

ValueError: Complex data not supported
[[[ 1.04670992e+01+0.00000000e+00j  9.89663315e+00+0.00000000e+00j
    9.87112141e+00+0.00000000e+00j ...  1.27105742e+01+0.00000000e+00j
    1.32443609e+01+0.00000000e+00j  1.42955847e+01+0.00000000e+00j]
  [-5.89109230e+00+2.13748804e-16j -4.51512146e+00-5.96832216e-01j
   -4.97937489e+00+5.87690234e-01j ... -6.65872622e+00+4.21497747e-02j
   -6.37788057e+00+3.87514114e-01j -7.37871408e+00+6.65729284e-01j]
  [ 1.13821411e+00-3.05762284e-17j -6.61318362e-01+2.69633383e-01j
   -8.59262645e-02-5.73369026e-01j ...  3.43340933e-01+5.78666985e-01j
   -1.33063987e-01-4.52870727e-01j -2.54739195e-01-6.12027943e-01j]
  ...
  [ 8.02112892e-02-2.38351866e-18j  1.25598192e-01+7.76425526e-02j
   -9.16602314e-02+4.97558057e-01j ... -3.21771026e-01-8.70816112e-01j
    2.04047680e-01-3.03144932e-01j  2.70314127e-01+8.84526610e-01j]
  [ 7.30685353e-01+6.72763925e-17j -8.55472505e-01+1.19729489e-01j
    2.85569370e-01-6.83539629e-01j ...  2.16854000e+00+6.4207494

In [50]:
mer=[]
cl_model_inp_1 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x = tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_1)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)

cl_model_inp_2 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x = tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_2)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)

cl_model_inp_3 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x=tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_3)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)

cl_model_inp_4 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x=tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_4)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)

cl_model_inp_5 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x=tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_5)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)

cl_model_inp_6 = tf.keras.layers.Input(shape=(denoised[0].shape[1],1))
x=tf.keras.layers.Conv1D(32, (3),activation='relu',strides =1,padding="same")(cl_model_inp_6)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
mer.append(x)



merge = tf.keras.layers.Concatenate(axis=-1)(mer)

x=tf.keras.layers.Conv1D(8, (3),activation='relu',strides =1,padding="same")(merge)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Conv1D(4, (3),activation='relu',strides =1,padding="same")(x)
x=tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Flatten()(x)

x=tf.keras.layers.Dense(200)(x)
x = tf.keras.layers.BatchNormalization()(x)
x =tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(4,activation='softmax')(x)

inp=[cl_model_inp_1,cl_model_inp_2,cl_model_inp_3,cl_model_inp_4,cl_model_inp_5,cl_model_inp_6]


model = tf.keras.Model(inputs=inp, outputs=output, name="final_model")
# lin_trainX.shape,lin_trainY.shape,lin_valX.shape,lin_valY.shape
# cnn_trainX.shape,cnn_trainY.shape,cnn_valX.shape,cnn_valY.shape
model.compile(optimizer='adam'  ,loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "final_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_68 (InputLayer)           [(None, 3872, 1)]    0                                            
__________________________________________________________________________________________________
input_69 (InputLayer)           [(None, 3872, 1)]    0                                            
__________________________________________________________________________________________________
input_70 (InputLayer)           [(None, 3872, 1)]    0                                            
__________________________________________________________________________________________________
input_71 (InputLayer)           [(None, 3872, 1)]    0                                            
________________________________________________________________________________________

In [51]:
# cl_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# cl_model.summary()

In [42]:
from tensorflow.keras.utils import to_categorical
output_matrix = to_categorical(clean_labels, num_classes = 4)


In [43]:
# train_clean = np.array(clean_pad[int(len(clean_pad)*0.8):])
# train_noise = np.array(noise_added[int(len(noise_added)*0.8):])
# test_clean = np.array(clean_pad[:int(len(clean_pad)*0.2)])
# test_noise = np.array(noise_added[:int(len(noise_added)*0.2)])

from sklearn.model_selection import train_test_split
F_X_train=[]
F_X_test=[]
for i in range(6):
    X_train, X_test,y_train, y_test = train_test_split(denoised[i],output_matrix ,
                                       random_state=98, 
                                       test_size=0.2, 
                                       shuffle=False)
    F_X_train.append(X_train)
    F_X_test.append(X_test)

((1648, 4), (412, 4), (6, 412, 3872, 1), (6, 1648, 3872, 1))

In [45]:
from tensorflow.keras.utils import plot_model
# plot_model(cl_model, to_file='cl_model.png', show_shapes=True, show_layer_names=True)
# F_y_train

In [52]:
history = model.fit(x=F_X_train,y=np.array(y_train),
                              epochs=150,
                              verbose=1,
                              batch_size=32,
                              validation_data=(F_X_test,np.array(y_test)))#,callbacks=[saver])

Epoch 1/150
52/52 [==============================] - 3s 58ms/step - loss: 1.9516 - acc: 0.3525 - val_loss: 2.9484 - val_acc: 0.0777
Epoch 2/150
52/52 [==============================] - 3s 50ms/step - loss: 1.2214 - acc: 0.5316 - val_loss: 3.2084 - val_acc: 0.0777
Epoch 3/150
52/52 [==============================] - 3s 50ms/step - loss: 0.9898 - acc: 0.6195 - val_loss: 3.8436 - val_acc: 0.0777
Epoch 4/150
42/52 [=======================>......] - ETA: 0s - loss: 0.8285 - acc: 0.6927

KeyboardInterrupt: 

In [38]:
import librosa.display
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

path = './data/voice/UNETPP_denoised/'
if not os.path.exists(path):
    os.mkdir(path)
for j in range(6):
    for i in tqdm(range(len(denoised[j]))):
        samp = np.reshape(denoised[j][i],(denoised[j][i].shape[0]))
        
#         y=clean_labels[i]
    #     path = './data/voice/mfcc/'+i+'.png'
        mfccs = librosa.stft(samp)
    #     mfccs = librosa.feature.mfcc(samp, sr=8000,n_mfcc=12)
        noise_stft_mag ,noisy_stft_phase=librosa.magphase(mfccs)
        noisy_signal_stft_unscaled_magnitude=librosa.amplitude_to_db(noise_stft_mag, ref=np.max)
        mfccs = (noisy_signal_stft_unscaled_magnitude)




        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        librosa.display.specshow(mfccs, sr=8000, x_axis='time',ax = ax)

        y=clean_labels[i]

        ax.axis('off')

        if not os.path.exists(path+str(y)):
            os.mkdir(path+str(y))
        fig.savefig(f'./{path+str(y)}/{j}_{i}.png',bbox_inches='tight',pad_inches = 0,dpi=200)

    

100%|██████████████████████████████████████████████████████████████████████████████| 2060/2060 [02:14<00:00, 15.28it/s]


In [46]:
fi = clf.feature_importances_
print(fi[:5])


[1. 0. 0. 0. 0.]


In [40]:
df = pd.read_csv('./openSmile_features.csv')
df = df.drop(['Unnamed: 0'],axis=1)

df_max_scaled = df.copy()
  
# apply normalization techniques
for column in df_max_scaled.columns:
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max()
df_max_scaled

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,0.659938,0.950,0.200,0.197933,0.534645,0.597947,0.707164,0.143530,0.691230,0.077533,...,0.361823,0.376466,0.281849,0.385656,-0.095628,0.631830,0.413775,0.426527,0.443604,0.270092
1,0.070858,0.550,0.825,0.636512,0.619101,0.602822,0.017669,0.011896,0.025302,0.712720,...,0.288643,0.683677,0.164192,0.162640,-0.488147,0.343609,0.190091,0.219038,0.248848,0.247192
2,0.069391,0.625,0.425,0.639501,0.621330,0.604547,0.016360,0.011170,0.023577,0.712033,...,0.185954,0.459081,0.281269,0.223266,0.428961,0.450694,0.192727,0.107676,0.349629,0.234263
3,0.227879,0.750,0.000,0.621882,0.614208,0.614837,0.036410,0.040705,0.067786,0.670634,...,0.386330,0.841254,0.220543,0.175467,0.436595,0.536413,0.300202,0.077010,0.356383,0.160833
4,0.045922,0.425,0.850,0.611246,0.592078,0.576797,0.011946,0.011871,0.020806,0.694598,...,0.092769,0.233441,0.280271,0.247058,0.904419,0.657624,0.393102,0.189578,0.423741,0.271425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,0.069895,0.350,0.000,0.574759,0.559773,0.545384,0.017463,0.011324,0.024588,0.638240,...,0.150160,0.598484,0.104375,0.104642,-0.431783,0.653581,0.176882,0.147270,0.203359,0.125287
2056,0.117417,0.475,0.900,0.567543,0.563548,0.559523,0.039401,0.029420,0.059217,0.596746,...,0.198739,0.422712,0.304712,0.240904,0.416297,0.417916,0.312550,0.153313,0.371247,0.130448
2057,0.079420,0.925,0.100,0.569066,0.554972,0.543270,0.018815,0.015643,0.029818,0.617115,...,0.152219,0.540996,0.170660,0.166203,-0.570425,0.564524,0.233315,0.093504,0.161253,0.218033
2058,0.132733,0.000,0.700,0.513993,0.539052,0.539221,0.094495,0.035064,0.107718,0.569116,...,0.202495,0.589538,0.107847,0.116438,-0.248013,0.514695,0.218165,0.171066,0.207127,0.127044


In [66]:
# df_max_scaled.to_csv('scaled_openSmile_features.csv')
# plt.scatter(df_max_scaled['audspec_lengthL1norm_sma_range'],clean_labels)
df

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1.720789,0.926829,0.195122,0.559211,1.574904,1.829900,1.015693,0.254996,1.270689,0.189148,...,2.772822,0.327405,1.238414,1.885932,-1.912556,0.537321,82.000000,51.920174,70.123120,30.298880
1,0.184763,0.536585,0.804878,1.798307,1.823685,1.844820,0.025378,0.021135,0.046512,1.738747,...,2.212009,0.594581,0.721445,0.795341,-9.762943,0.292212,37.671272,26.663046,39.336920,27.730024
2,0.180936,0.609756,0.414634,1.806754,1.830251,1.850096,0.023497,0.019845,0.043342,1.737070,...,1.425056,0.399254,1.235868,1.091814,8.579215,0.383280,38.193750,13.107221,55.268005,26.279620
3,0.594195,0.731707,0.000000,1.756976,1.809271,1.881587,0.052295,0.072316,0.124611,1.636073,...,2.960627,0.731623,0.969043,0.858066,8.731894,0.456176,59.492546,9.374274,56.335660,18.042300
4,0.119742,0.414634,0.829268,1.726926,1.744083,1.765173,0.017157,0.021090,0.038247,1.694537,...,0.710935,0.203020,1.231484,1.208162,18.088370,0.559257,77.903080,23.076878,66.983284,30.448496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,0.182250,0.341463,0.000000,1.623841,1.648923,1.669042,0.025081,0.020119,0.045200,1.557046,...,1.150750,0.520491,0.458614,0.511721,-8.635655,0.555819,35.053720,17.926868,32.146156,14.054689
2056,0.306164,0.463415,0.878049,1.603453,1.660044,1.712312,0.056591,0.052268,0.108859,1.455817,...,1.523034,0.367625,1.338874,1.178066,8.325937,0.355404,61.939760,18.662441,58.685272,14.633716
2057,0.207087,0.902439,0.097561,1.607757,1.634780,1.662571,0.027023,0.027791,0.054814,1.505510,...,1.166528,0.470494,0.749862,0.812767,-11.408493,0.480083,46.237250,11.382081,25.490156,24.458970
2058,0.346101,0.000000,0.682927,1.452162,1.587885,1.650180,0.135723,0.062295,0.198017,1.388412,...,1.551818,0.512710,0.473869,0.569402,-4.960259,0.437707,43.234997,20.823532,32.741800,14.251777


15071.543

In [64]:
feat = []
for i in tqdm(range(len(denoised))):
    stft = librosa.stft(np.reshape(denoised[i],(denoised[i].shape[0])))
    spec = librosa.power_to_db(np.abs(stft)**2)
    l1_norm = np.linalg.norm(spec, ord=1, axis=0)
    range_l1_norm = np.ptp(l1_norm)
    feat.append(range_l1_norm)

100%|████████████████████████████████████████████████████████████████████████████| 2060/2060 [00:01<00:00, 1130.13it/s]


In [65]:
scipy.stats.spearmanr(feat, b=clean_labels)

SpearmanrResult(correlation=-0.07793842856206307, pvalue=0.00039914518147156006)